In [1]:
import os
import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "portfolio_management.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

from common.models import Brokers, Prices, Transactions, FX, Assets
from users.models import CustomUser
from utils import NAV_at_date, Irr, chart_dates, chart_labels, chart_colour, portfolio_at_date, calculate_security_nav, currency_format, calculate_portfolio_value
from datetime import date
from django.db.models import F, Sum

In [2]:
selected_brokers = broker_ids = [2]
effective_current_date = date.today()
currency_target = 'USD'
number_of_digits = 2

CustomUser.objects.get(id=2)

<CustomUser: Yaroslav>

In [3]:
security_id = 7
security = Assets.objects.get(id=security_id)
security.price_at_date(effective_current_date).price

quote = security.prices.filter(date__lte=effective_current_date).order_by('-date').first()
print(currency_target)
if currency_target is not None:
    quote.price = quote.price * FX.get_rate(security.currency, currency_target, effective_current_date)['FX']
quote

USD


<Prices: Long/short is at 23.972000000000 on 2020-12-01>

In [4]:
from django.db.models import Count

duplicates = (
            Prices.objects.values('date', 'security', 'price')
            .annotate(count_id=Count('id'))
            .filter(count_id__gt=1)
        )

total_duplicates = len(duplicates)
print((f"Found {total_duplicates} duplicate entries."))

for i, duplicate in enumerate(duplicates, start=1):
            # Keep one entry and delete the rest
            entries = Prices.objects.filter(
                date=duplicate['date'],
                security=duplicate['security'],
                price=duplicate['price']
            )
            entries_to_keep = entries.first()
            entries.exclude(id=entries_to_keep.id).delete()

            # Print status update
            print(f"Duplicates are being removed: {i} out of {total_duplicates}\r", end='', flush=True)
            
print("\nDuplicates removed.")


Found 0 duplicate entries.

Duplicates removed.


In [5]:
import datetime
from decimal import Decimal


def end_of_year_price_correction(user, year, broker_name, target_nav, asset_name):
    
    target_nav = round(Decimal(target_nav), 2)
    
    # Get the broker
    try:
        broker = Brokers.objects.get(name=broker_name)
    except Brokers.DoesNotExist:
        return {"error": f"Broker {broker_name} does not exist."}

    # Get the asset
    try:
        asset = Assets.objects.get(name=asset_name)
    except Assets.DoesNotExist:
        return {"error": f"Asset {asset_name} does not exist."}

    # Calculate end of year date
    end_of_year_date = datetime.date(year, 12, 31)

    # Fetch NAV at the end of the year
    # nav_at_end_of_year = get_nav_at_date(broker, end_of_year_date)
    nav_at_end_of_year = NAV_at_date(user.id, [broker.id], end_of_year_date, asset.currency, [])['Total NAV']
    if nav_at_end_of_year is None:
        return {"error": f"No NAV found for broker {broker_name} at the end of {year}."}

    # Fetch asset price and position at the end of the year
    price_at_end_of_year = asset.price_at_date(end_of_year_date)
    if not price_at_end_of_year:
        return {"error": f"No price found for asset {asset_name} at the end of {year}."}

    position_at_end_of_year = asset.position(end_of_year_date, [broker.id])

    # Calculate new price
    old_price = price_at_end_of_year.price
    new_price = old_price + ((target_nav - nav_at_end_of_year) / position_at_end_of_year)

    # Display information
    old_asset_value = round(Decimal(old_price * position_at_end_of_year), 2)
    new_asset_value = round(Decimal(new_price * position_at_end_of_year), 2)

    result = {
        "old_price": old_price,
        "new_price": new_price,
        "old_asset_value": old_asset_value,
        "new_asset_value": new_asset_value,
        "old_nav": nav_at_end_of_year,
        "target_nav": target_nav
    }

    print(f"Old Price: {result['old_price']}, New Price: {result['new_price']}")
    print(f"Old Asset Value: {result['old_asset_value']}, New Asset Value: {result['new_asset_value']}")
    print(f"Old NAV: {result['old_nav']}, Target NAV: {result['target_nav']}")

    # Ask for confirmation
    confirm = input("Do you want to update the price? (yes/no): ")

    if confirm.lower() == 'yes':
        # Update the price
        price_instance, created = Prices.objects.get_or_create(
            security=asset, date=end_of_year_date,
            defaults={'price': new_price}
        )
        if not created:
            price_instance.price = new_price
            price_instance.save()

        result["status"] = "Price updated successfully."
        print("New price is:", price_instance.price)
    else:
        result["status"] = "Price update canceled."

    return result

user = CustomUser.objects.filter(username='Yaroslav').first()
year = 2013
broker_name = 'UBS Pension'
target_nav = 29444.38
asset_name = 'Emerging Markets Equity Fund'

end_of_year_price_correction(user, year, broker_name, target_nav, asset_name)

Old Price: 0.936632, New Price: 0.9366287795154500008662569358
Old Asset Value: 7983.28, New Asset Value: 7983.26
Old NAV: 29444.407449461779, Target NAV: 29444.38


{'old_price': Decimal('0.936632'),
 'new_price': Decimal('0.9366287795154500008662569358'),
 'old_asset_value': Decimal('7983.28'),
 'new_asset_value': Decimal('7983.26'),
 'old_nav': Decimal('29444.407449461779'),
 'target_nav': Decimal('29444.38'),
 'status': 'Price update canceled.'}

In [6]:
from utils import import_asset_prices_from_csv

# file_path = "../Data/Pricing data from Galaxy.csv"
investor_id = 2

# import_asset_prices_from_csv(file_path, investor_id)

In [40]:
import pandas as pd
from fuzzywuzzy import process
from datetime import datetime
from django.db.models import Q
import sys

def parse_charles_stanley_transactions(file, currency, broker_id, investor_id):
    df = pd.read_excel(file, header=3)
    transactions = []

    # print(df)

    broker = Brokers.objects.get(id=broker_id)
    investor = CustomUser.objects.get(id=investor_id)

    # quantity_field = Transactions._meta.get_field('quantity')
    quantity_decimal_places = 4
    # price_field = Transactions._meta.get_field('price')
    price_decimal_places = 5

    # Cache to store user-defined security mappings
    security_cache = {}

    def find_or_prompt_security(stock_description, investor, broker, security_cache):

        # Check if we have a cached mapping
        if stock_description in security_cache:
            return security_cache[stock_description]
        
        def find_best_matching_security(description):
            securities = Assets.objects.filter(
                Q(investor=investor) & Q(brokers=broker)
            )
            security_names = [security.name for security in securities]
            best_match = process.extractOne(description, security_names, score_cutoff=60)
            
            if best_match:
                match_name, match_score = best_match
                print(f"Potential match found: '{match_name}' (Similarity: {match_score}%)")
                sys.stdout.flush()
                user_confirm = input(f"Do you agree with this match for '{description}'? (yes/no/skip/exit): ").lower()
                
                if user_confirm == 'yes':
                    matched_security = securities.get(name=match_name)
                    security_cache[stock_description] = matched_security  # Update cache
                    print(f"Match confirmed and cached for future use.")
                    return matched_security
                elif user_confirm == 'exit':
                    raise KeyboardInterrupt("User requested to exit")
                elif user_confirm == 'skip':
                    print("Skipping this transaction.")
                    security_cache[stock_description] = None  # Cache the skip decision
                    return None
                else:
                    print("Match rejected.")
                    return None
            return None

        security = find_best_matching_security(stock_description)
        
        if security is None:
            print(f"No matching security found for '{stock_description}'.")
            sys.stdout.flush()  # Ensure print is displayed
            user_input = input("Please enter the correct security name, 'skip' to skip this transaction, or 'exit' to stop processing: ")
            
            if user_input.lower() == 'exit':
                raise KeyboardInterrupt("User requested to exit")
            elif user_input.lower() == 'skip':
                print("Skipping this transaction.")
                security_cache[stock_description] = None
            elif user_input:
                # Check for an exact match with the user input
                try:
                    security = Assets.objects.get(
                        Q(investor=investor) & Q(brokers=broker) & Q(name__iexact=user_input)
                    )
                    print(f"Security '{security.name}' found and selected.")
                     # Cache the user-defined mapping
                    security_cache[stock_description] = security
                except Assets.DoesNotExist:
                    print(f"No exact match found for '{user_input}'. This transaction will be skipped.")
                    security_cache[stock_description] = None
            else:
                print("No security name provided. This transaction will be skipped.")
                security_cache[stock_description] = None

        return security

    for index, row in df.iterrows():
        
        date_str = row['Date']

        # Check if there is any data in the current row
        if pd.notna(date_str):
            date = datetime.strptime(date_str, '%d-%b-%Y').date()
            description = row['Description']
            stock_description = row['Stock Description']
            price = row['Price']
            debit = row['Debit']
            credit = row['Credit']

            if description == '* BALANCE B/F *' or description == 'Cash Transfers ISA':
                print(f'Skipped: {description}')
                continue
            elif description == 'Funds Platform Fee':
                transaction_data = {
                    'investor': investor,
                    'broker': broker,
                    'currency': currency,
                    'type': 'Broker commission',
                    'date': date,
                    'commission': round(-Decimal(str(debit)), 2),
                }
                transactions.append(transaction_data)

            elif description == 'Stocks & Shares Subs' or 'ISA Subscription' in description:
                transaction_data = {
                    'investor': investor,
                    'broker': broker,
                    'currency': currency,
                    'type': 'Cash in',
                    'date': date,
                    'cash_flow': Decimal(str(credit)),
                }
                transactions.append(transaction_data)

            elif pd.notna(stock_description):

                security = find_or_prompt_security(stock_description, investor, broker, security_cache)
                if security is None:
                    continue # Skip this transaction

                transaction_type = 'Buy' if debit > 0 else 'Sell'
                price = round(Decimal(str(price)), price_decimal_places)
                if transaction_type == 'Buy':
                    quantity = round(Decimal(str(debit)) / price, quantity_decimal_places)
                else:
                    quantity = round(-Decimal(str(credit)) / price, quantity_decimal_places)

                transaction_data = {
                    'investor': investor,
                    'broker': broker,
                    'security': security,
                    'currency': currency,
                    'type': transaction_type,
                    'date': date,
                    'quantity': quantity,
                    'price': price,
                }

                transactions.append(transaction_data)
            else:
                print(f'Skipped: {description}')

                # print(f"Processed row {index + 1} of {len(df)}")
                # sys.stdout.flush()  # Ensure print is displayed
        # except Exception as e:
        #     print(f"Error processing row {index}: {e}")
        #     continue

    if transactions:
        print(f"\nProcessed {len(transactions)} transactions.")
        save_choice = input(f"Do you want to save these transactions for {broker.name}? (yes/no): ").lower()
        if save_choice == 'yes':
            Transactions.objects.bulk_create([Transactions(**data) for data in transactions])
            print("Transactions saved to the database.")
        else:
            print("Transactions were not saved to the database.")
    else:
        print("No transactions to save.")

    return transactions

file = "../Data/JISA_LL_4682719_Generated_1308_04Aug.xlsx"
currency = 'GBP'
broker_id = 9
investor_id = 2

t = parse_charles_stanley_transactions(file, currency, broker_id, investor_id)
t

Potential match found: 'HSBC FTSE 250 Index Accumulation C' (Similarity: 72%)
Match confirmed and cached for future use.
Potential match found: 'Fidelity Index US Fund P Accumulation' (Similarity: 70%)
Match confirmed and cached for future use.
No matching security found for 'FIL INV SVCS UK INDEX US P ACC NAV'.
Security 'Fidelity Index US Fund P Accumulation' found and selected.
Potential match found: 'HSBC FTSE 250 Index Accumulation C' (Similarity: 60%)
Match confirmed and cached for future use.
Potential match found: 'Vanguard U.S. Equity Index Fund GBP Acc' (Similarity: 75%)
Match confirmed and cached for future use.
Potential match found: 'HSBC FTSE 250 Index Accumulation C' (Similarity: 62%)
Match confirmed and cached for future use.
Skipped: Gross interest to 31/03/15
Skipped: Cash Transfers ISA
Skipped: Cash Transfers ISA
Skipped: * BALANCE B/F *

Processed 166 transactions.
Transactions saved to the database.


[{'investor': <CustomUser: Yaroslav>,
  'broker': <Brokers: Charles Stanley - JISA - LL>,
  'currency': 'GBP',
  'type': 'Broker commission',
  'date': datetime.date(2024, 8, 1),
  'commission': Decimal('-11.44')},
 {'investor': <CustomUser: Yaroslav>,
  'broker': <Brokers: Charles Stanley - JISA - LL>,
  'currency': 'GBP',
  'type': 'Broker commission',
  'date': datetime.date(2024, 7, 1),
  'commission': Decimal('-10.88')},
 {'investor': <CustomUser: Yaroslav>,
  'broker': <Brokers: Charles Stanley - JISA - LL>,
  'currency': 'GBP',
  'type': 'Broker commission',
  'date': datetime.date(2024, 6, 3),
  'commission': Decimal('-11.08')},
 {'investor': <CustomUser: Yaroslav>,
  'broker': <Brokers: Charles Stanley - JISA - LL>,
  'currency': 'GBP',
  'type': 'Broker commission',
  'date': datetime.date(2024, 5, 1),
  'commission': Decimal('-10.45')},
 {'investor': <CustomUser: Yaroslav>,
  'broker': <Brokers: Charles Stanley - JISA - LL>,
  'currency': 'GBP',
  'type': 'Broker commission'